In [ ]:
# Imports
import h5py
import importlib
import os
import sys
import time


Applying all parameter functions on to the first 4 training data sets.

In [23]:
# Get the base directory of the repository
repo_base_dir = os.getcwd()

# Import parameter extraction functions
from parameter_functions.tdrift import tdrift
from parameter_functions.rea import rea
from parameter_functions.dcr import find_dcr
from parameter_functions.peakandtailslope import extract_peak_and_tail_slope
from parameter_functions.current_amplitude import max_amplitude
from parameter_functions.fourier_lfpr import normalized_fourier, lfpr
from parameter_functions.lq80 import LQ80
from parameter_functions.agr import area_growth_rate
from parameter_functions.inflection import inflection_points
from parameter_functions.rising_edge import rising_edge_slope

# Get all HDF5 files in the data directory
data_dir = os.path.join(repo_base_dir, "data/") 
data_files = [f for f in os.listdir(str(data_dir)) if f.endswith(".hdf5")]

# %%
start = time.time()
# Parameter extraction from all waveform data
results = []
for file in data_files:
    with h5py.File(data_dir + file, 'r') as f:
        for wave in np.arange(f['raw_waveform'].shape[0]):
            # Needed labels
            waveform = f['raw_waveform'][wave]
            tp0 = f['tp0'][wave]
            id = f['id'][wave]

            # Parameter extraction
            tdriftVal = tdrift(waveform, tp0) # 99.9%, 50%, 10%
            reaVal = rea(waveform, tp0)
            dcrVal = find_dcr(waveform)
            peakandtailVal = extract_peak_and_tail_slope(waveform)
            max_amp = max_amplitude(waveform)
            lfprVal = lfpr(normalized_fourier(waveform, tp0)[0])
            lq80 = LQ80(waveform, tp0)
            agr = area_growth_rate(waveform)
            inflectionpts = inflection_points(waveform, tp0)
            res = rising_edge_slope(waveform, tp0)

            results.append({'id': id,
                'tdrift': tdriftVal[0],
                'tdrift50': tdriftVal[1],
                'tdrift10': tdriftVal[2],
                'rea': reaVal,
                'dcr': dcrVal,
                'peakindex': peakandtailVal[0],
                'peakvalue': peakandtailVal[1],
                'tailslope': peakandtailVal[2],
                'currentamp': max_amp,
                'lfpr': lfprVal,
                'lq80': lq80,
                'areagrowthrate': agr,
                'inflection point': inflectionpts,
                'risingedgeslope': res
            })
        df = pd.DataFrame(results)
    df.to_csv("results.csv", index=False)
end = time.time()
print(end - start)


/Users/atlus0927/Desktop/School/DSC 180A/code files/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1177: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
/Users/atlus0927/Desktop/School/DSC 180A/code files/venv/lib/python3.9/site-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/atlus0927/Desktop/School/DSC 180A/code files/venv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/atlus0927/Desktop/School/DSC 180A/code files/venv/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


958.4983899593353
